In [1]:
"""Module for fetching data from the SEC EDGAR Archives"""
import json
import os
import re
import requests
from typing import List, Optional, Tuple, Union
import sys

if sys.version_info < (3, 8):
    from typing_extensions import Final
else:
    from typing import Final

import webbrowser

from ratelimit import limits, sleep_and_retry

VALID_FILING_TYPES: Final[List[str]] = [
    "10-K",
    "10-Q",
    "S-1",
    "10-K/A",
    "10-Q/A",
    "S-1/A",
]

SEC_ARCHIVE_URL: Final[str] = "https://www.sec.gov/Archives/edgar/data"
SEC_SEARCH_URL: Final[str] = "http://www.sec.gov/cgi-bin/browse-edgar"
SEC_SUBMISSIONS_URL = "https://data.sec.gov/submissions"


def get_filing(
    cik: Union[str, int], accession_number: Union[str, int], company: str, email: str
) -> str:
    """Fetches the specified filing from the SEC EDGAR Archives. Conforms to the rate
    limits specified on the SEC website.
    ref: https://www.sec.gov/os/accessing-edgar-data"""
    session = _get_session(company, email)
    return _get_filing(session, cik, accession_number)


@sleep_and_retry
@limits(calls=10, period=1)
def _get_filing(
    session: requests.Session, cik: Union[str, int], accession_number: Union[str, int]
) -> str:
    """Wrapped so filings can be retrieved with an existing session."""
    url = archive_url(cik, accession_number)
    response = session.get(url)
    response.raise_for_status()
    return response.text


@sleep_and_retry
@limits(calls=10, period=1)
def get_cik_by_ticker(session: requests.Session, ticker: str) -> str:
    """Gets a CIK number from a stock ticker by running a search on the SEC website."""
    cik_re = re.compile(r".*CIK=(\d{10}).*")
    url = _search_url(ticker)
    response = session.get(url, stream=True)
    response.raise_for_status()
    results = cik_re.findall(response.text)
    return str(results[0])


@sleep_and_retry
@limits(calls=10, period=1)
def get_forms_by_cik(session: requests.Session, cik: Union[str, int]) -> dict:
    """Gets retrieves dict of recent SEC form filings for a given cik number."""
    json_name = f"CIK{cik}.json"
    response = session.get(f"{SEC_SUBMISSIONS_URL}/{json_name}")
    response.raise_for_status()
    content = json.loads(response.content)
    recent_forms = content["filings"]["recent"]
    form_types = {k: v for k, v in zip(recent_forms["accessionNumber"], recent_forms["form"])}
    return form_types


def _get_recent_acc_num_by_cik(
    session: requests.Session, cik: Union[str, int], form_types: List[str]
) -> Tuple[str, str]:
    """Returns accession number and form type for the most recent filing for one of the
    given form_types (AKA filing types) for a given cik."""
    retrieved_form_types = get_forms_by_cik(session, cik)
    for acc_num, form_type_ in retrieved_form_types.items():
        if form_type_ in form_types:
            return _drop_dashes(acc_num), form_type_
    raise ValueError(f"No filings found for {cik}, looking for any of: {form_types}")


def get_recent_acc_by_cik(
    cik: str,
    form_type: str,
    company: Optional[str] = None,
    email: Optional[str] = None,
) -> Tuple[str, str]:
    """Returns (accession_number, retrieved_form_type) for the given cik and form_type.
    The retrieved_form_type may be an amended version of requested form_type, e.g. 10-Q/A for 10-Q.
    """
    session = _get_session(company, email)
    return _get_recent_acc_num_by_cik(session, cik, _form_types(form_type))


def get_recent_cik_and_acc_by_ticker(
    ticker: str,
    form_type: str,
    company: Optional[str] = None,
    email: Optional[str] = None,
) -> Tuple[str, str, str]:
    """Returns (cik, accession_number, retrieved_form_type) for the given ticker and form_type.
    The retrieved_form_type may be an amended version of requested form_type, e.g. 10-Q/A for 10-Q.
    """
    session = _get_session(company, email)
    cik = get_cik_by_ticker(session, ticker)
    acc_num, retrieved_form_type = _get_recent_acc_num_by_cik(session, cik, _form_types(form_type))
    return cik, acc_num, retrieved_form_type


def get_form_by_ticker(
    ticker: str,
    form_type: str,
    allow_amended_filing: Optional[bool] = True,
    company: Optional[str] = None,
    email: Optional[str] = None,
) -> str:
    """For a given ticker, gets the most recent form of a given form_type."""
    session = _get_session(company, email)
    cik = get_cik_by_ticker(session, ticker)
    return get_form_by_cik(
        cik, form_type, allow_amended_filing=allow_amended_filing, company=company, email=email
    )


def _form_types(form_type: str, allow_amended_filing: Optional[bool] = True):
    """Potentialy expand to include amended filing, e.g.:
    "10-Q" -> "10-Q/A"
    """
    assert form_type in VALID_FILING_TYPES
    if allow_amended_filing and not form_type.endswith("/A"):
        return [form_type, f"{form_type}/A"]
    else:
        return [form_type]


def get_form_by_cik(
    cik: str,
    form_type: str,
    allow_amended_filing: Optional[bool] = True,
    company: Optional[str] = None,
    email: Optional[str] = None,
) -> str:
    """For a given CIK, returns the most recent form of a given form_type. By default
    an amended version of the form_type may be retrieved (allow_amended_filing=True).
    E.g., if form_type is "10-Q", the retrived form could be a 10-Q or 10-Q/A.
    """
    session = _get_session(company, email)
    acc_num, _ = _get_recent_acc_num_by_cik(
        session, cik, _form_types(form_type, allow_amended_filing)
    )
    text = _get_filing(session, cik, acc_num)
    return text


def open_form(cik, acc_num):
    """For a given cik and accession number, opens the index page in default browser for the
    associated SEC form"""
    acc_num = _drop_dashes(acc_num)
    webbrowser.open_new_tab(f"{SEC_ARCHIVE_URL}/{cik}/{acc_num}/{_add_dashes(acc_num)}-index.html")


def open_form_by_ticker(
    ticker: str,
    form_type: str,
    allow_amended_filing: Optional[bool] = True,
    company: Optional[str] = None,
    email: Optional[str] = None,
):
    """For a given ticker, opens the index page in default browser for the most recent form of a
    given form_type."""
    session = _get_session(company, email)
    cik = get_cik_by_ticker(session, ticker)
    acc_num, _ = _get_recent_acc_num_by_cik(
        session, cik, _form_types(form_type, allow_amended_filing)
    )
    open_form(cik, acc_num)


def archive_url(cik: Union[str, int], accession_number: Union[str, int]) -> str:
    """Builds the archive URL for the SEC accession number. Looks for the .txt file for the
    filing, while follows a {accession_number}.txt format."""
    filename = f"{_add_dashes(accession_number)}.txt"
    accession_number = _drop_dashes(accession_number)
    return f"{SEC_ARCHIVE_URL}/{cik}/{accession_number}/{filename}"


def _search_url(cik: Union[str, int]) -> str:
    search_string = f"CIK={cik}&Find=Search&owner=exclude&action=getcompany"
    url = f"{SEC_SEARCH_URL}?{search_string}"
    return url


def _add_dashes(accession_number: Union[str, int]) -> str:
    """Adds the dashes back into the accession number"""
    accession_number = str(accession_number)
    return f"{accession_number[:10]}-{accession_number[10:12]}-{accession_number[12:]}"


def _drop_dashes(accession_number: Union[str, int]) -> str:
    """Converts the accession number to the no dash representation."""
    accession_number = str(accession_number).replace("-", "")
    return accession_number.zfill(18)


def _get_session(company: Optional[str] = None, email: Optional[str] = None) -> requests.Session:
    """Creates a requests sessions with the appropriate headers set. If these headers are not
    set, SEC will reject your request.
    ref: https://www.sec.gov/os/accessing-edgar-data"""
    if company is None:
        company = os.environ.get("SEC_API_ORGANIZATION")
    if email is None:
        email = os.environ.get("SEC_API_EMAIL")
    assert company
    assert email
    session = requests.Session()
    session.headers.update(
        {
            "User-Agent": f"{company} {email}",
            "Content-Type": "text/html",
        }
    )
    return session


In [2]:
import json
import os
import re
import requests
from typing import List, Optional, Tuple, Union
import sys

In [2]:
text = get_form_by_ticker(
    'aapl',
    '10-K',
    company='Unstructured Technologies',
    email='support@unstructured.io'
)

## Cleaning up the .XML file

In [5]:
from unstructured.documents.html import HTMLDocument

html_document = HTMLDocument.from_string(text).doc_after_cleaners(skip_headers_and_footers=True, skip_table_text=True)


In [7]:
html_document.pages[0].elements[::]

### Check to see if a section could be a title

In [8]:
from unstructured.nlp.partition import is_possible_title

#Getting the name of a title from the HTMLTitle object
text_title = html_document.pages[0].elements[22].text

print(text_title)

#Check if that title could be a title or not
is_possible_title(text_title)


Indicate by check mark if the Registrant is not required to file reports pursuant to Section 13 or Section 15(d) of the Act.


False

### Making bricks

In [9]:
import re
from unstructured.documents.elements import Title

In [10]:
ITEM_TITLE_RE = re.compile(
    r"(?i)item \d{1,3}(?:[a-z]|\([a-z]\))?(?:\.)?(?::)?"
)

In [11]:
def is_10k_item_title(title: str) -> bool:
    """Determines if a title corresponds to a 10-K item heading."""
    return ITEM_TITLE_RE.match(title) is not None

In [12]:
for element in html_document.elements:
    if isinstance(element, Title) and is_10k_item_title(element.text):
        print(element)



Item 1.    Business
Item 1A.    Risk Factors
Item 1B.    Unresolved Staff Comments
Item 2.    Properties
Item 3.    Legal Proceedings
Item 4.    Mine Safety Disclosures
Item 7.    Management’s Discussion and Analysis of Financial Condition and Results of Operations
Item 7A.    Quantitative and Qualitative Disclosures About Market Risk
Item 8.    Financial Statements and Supplementary Data
Item 9.    Changes in and Disagreements with Accountants on Accounting and Financial Disclosure
Item 9A.    Controls and Procedures
Item 9B.    Other Information
Item 9C.    Disclosure Regarding Foreign Jurisdictions that Prevent Inspections
Item 10.    Directors, Executive Officers and Corporate Governance
Item 11.    Executive Compensation
Item 13.    Certain Relationships and Related Transactions, and Director Independence
Item 14.    Principal Accountant Fees and Services
Item 15.    Exhibit and Financial Statement Schedules
Item 16.    Form 10-K Summary


### Cleaning bricks

In [13]:
from unstructured.cleaners.core import clean_extra_whitespace

In [14]:
titles_names = []
titles = []
for element in html_document.elements:
    element.text = clean_extra_whitespace(element.text)
    if isinstance(element, Title) and is_10k_item_title(element.text):
        titles.append(element)
        titles_names.append(element.text)
        print(element)

print(titles_names)

Item 1. Business
Item 1A. Risk Factors
Item 1B. Unresolved Staff Comments
Item 2. Properties
Item 3. Legal Proceedings
Item 4. Mine Safety Disclosures
Item 7. Management’s Discussion and Analysis of Financial Condition and Results of Operations
Item 7A. Quantitative and Qualitative Disclosures About Market Risk
Item 8. Financial Statements and Supplementary Data
Item 9. Changes in and Disagreements with Accountants on Accounting and Financial Disclosure
Item 9A. Controls and Procedures
Item 9B. Other Information
Item 9C. Disclosure Regarding Foreign Jurisdictions that Prevent Inspections
Item 10. Directors, Executive Officers and Corporate Governance
Item 11. Executive Compensation
Item 13. Certain Relationships and Related Transactions, and Director Independence
Item 14. Principal Accountant Fees and Services
Item 15. Exhibit and Financial Statement Schedules
Item 16. Form 10-K Summary
['Item 1. Business', 'Item 1A. Risk Factors', 'Item 1B. Unresolved Staff Comments', 'Item 2. Propert

In [15]:
section_info = []
for i, el in enumerate(html_document.elements):
    if el.id == titles[0].id:
        break
first_title_index = i
for i in range(first_title_index, first_title_index+30):
    section_info.append(html_document.elements[i].text)


In [16]:
#{type(el) for el in html_document.elements}

### Using OpenAI

In [3]:
import os
import openai

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]


In [18]:
def find_relevant_section(question, list_elements):
    """Use Open API to attempt to find what section is most relevant to answering a given question

    Args:
        question (String): The question to ask GPT
        list_elements (list): A list of all the titles in the 10-K

    Returns:
        index (integer): The index in the list of the most relevant section
    """

    human_template = f"""You are helping me answer questions about a 10-K document for me. Enclosed by the word START and END, I will give you a list of section titles from a 10-K. Each item in the list is a different section.
    I will then give you a question, and you must tell me which section I am most likely to find the answer to that question.

    Here is the 10-K data:
    START
    {list_elements}
    END

    This is the question: {question}

    Which section is most relevant? Please only give me no text, and only give me the index of the section, with the first item of the list being index 0, that is the most relevant as a single integer. Give me a 1 or 2 digit integer.
    """

    human_message_prompt = {"role": "user", "content": human_template}


    out = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        temperature=0,
        messages=[
            human_message_prompt,
        ],
        api_key=OPENAI_API_KEY
    )
    output = out["choices"][0]["message"]["content"]

    return output

In [19]:
def get_answer(question, index):
    """Use Open API to attempt to get an answer to a question based off one section in the 10-K

    Args:
        question (String): The question to ask GPT
        index (int): The index of the most relevant section

    Returns:
        answer (String): GPT's answer to the question
    """

    #Getting the relevant section
    section_info = []
    start_index_elements = 0
    end_index_elements = len(html_document.elements)
    for i, el in enumerate(html_document.elements):
        if el.id == titles[index].id:
            start_index_elements = i
        if el.id == titles[index+1].id:
            end_index_elements = i
            break
    for i in range(start_index_elements, end_index_elements):
        section_info.append(html_document.elements[i].text)

    human_template = f"""You are analyzing a 10-K document for me. Enclosed by the word START and END, I will give you the data of the 10-K and you will answer a question based ONLY on information you find enclosed in the triple quotes, so you will use no outside information.
    If you cannot find the answer to the question in the information provided, please tell me that you were unable to find an answer.

    Here is the 10-K data:
    START
    {section_info}
    END

    This is the question: {question}
    """

    human_message_prompt = {"role": "user", "content": human_template}


    out = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        temperature=0,
        messages=[
            human_message_prompt,
        ],
        api_key=OPENAI_API_KEY
    )
    answer = out["choices"][0]["message"]["content"]

    return answer


In [20]:
def ask_gpt_question(question):
    """Uses the two functions to find the index, then query GPT to answer the question.

    Args:
        question (String): The question to ask GPT

    Returns:
        answer (String): GPT's answer to the question
    """


    index = int(find_relevant_section(question, titles_names))

    return get_answer(question, index)


### Testing our function by asking it some questions

In [21]:
print(ask_gpt_question("How does the company generate its revenue? What are its main products or services?"))

The company generates its revenue through the sale of various products and services. Its main products include iPhone, Mac, iPad, and Wearables, Home and Accessories. Its main services include advertising, AppleCare, cloud services, digital content, payment services, and other services.


In [22]:
print(ask_gpt_question("What are the primary risks and uncertainties the company faces? How might they impact its future performance?"))

The primary risks and uncertainties the company faces include:

1. Macroeconomic and Industry Risks: Adverse economic conditions, such as inflation, slower growth or recession, new or increased tariffs, changes to fiscal and monetary policy, tighter credit, higher interest rates, high unemployment, and currency fluctuations can adversely impact consumer confidence and spending, leading to a decline in demand for the company's products and services. Uncertainty and decline in global or regional economic conditions can also impact the company's suppliers, contract manufacturers, logistics providers, distributors, and other channel partners, leading to financial instability, reduced liquidity, and declines in the fair value of the company's financial instruments.

2. Impact of COVID-19 Pandemic: The COVID-19 pandemic has significantly impacted global economic activity and caused volatility and disruption in global financial markets. The pandemic has disrupted the company's supply chain, r

In [23]:
print(ask_gpt_question("What are the company's strategic priorities and growth prospects?"))

I am sorry, but I was unable to find an answer to your question in the provided information.


In [24]:
print(ask_gpt_question("Does the company operate in multiple geographic regions or industries?"))

I'm sorry, but I was unable to find an answer to your question in the provided information.


In [25]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def compare_strings(text1, text2):
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([text1, text2])
    # Calculate the cosine similarity between the vectors
    similarity = cosine_similarity(vectors)
    print(similarity)


In [26]:
print(compare_strings("hello", "hello"))

[[1. 1.]
 [1. 1.]]
None


## Trying to fine tune the model

In [4]:
#openai.api_key = os.getenv("OPENAI_API_KEY")

openai.api_key = os.environ["OPENAI_API_KEY"]

In [6]:
#WORKING VERSION
#response = openai.File.create(
#  file=open("newtest.jsonl", "rb"),
#  purpose='fine-tune',
#  user_provided_filename="train"
#)

response = openai.File.create(
  file=open("./openaitest2.jsonl", "rb"),
  purpose='fine-tune',
  user_provided_filename="train"
)

file_id = response['id']
print(f"File successfully uploaded with ID: {file_id}")


File successfully uploaded with ID: file-G66jFKJJLR4twLoGEfQsTV9M


In [7]:
#check file status
openai.FineTuningJob.list()

<OpenAIObject list at 0x110319630> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job",
      "id": "ftjob-0HFEZZwlI6vi3qWZeyhg20q9",
      "model": "gpt-3.5-turbo-0613",
      "created_at": 1696364522,
      "finished_at": 1696372683,
      "fine_tuned_model": "ft:gpt-3.5-turbo-0613:personal:sarcastic-test:85iJgAnK",
      "organization_id": "org-WM0C8DmpfiXb4ySIeUGFLcf2",
      "result_files": [
        "file-IsvRuCmyqbnsDU0LyU2KGDxy"
      ],
      "status": "succeeded",
      "validation_file": null,
      "training_file": "file-Sa73VHGZQEbjcYJUyeC5A83W",
      "hyperparameters": {
        "n_epochs": 10
      },
      "trained_tokens": 4670,
      "error": null
    },
    {
      "object": "fine_tuning.job",
      "id": "ftjob-yHdajmDdWXOXYX5DdAXtFPBc",
      "model": "gpt-3.5-turbo-0613",
      "created_at": 1696363691,
      "finished_at": 1696371854,
      "fine_tuned_model": "ft:gpt-3.5-turbo-0613:personal::85i6JinG",
      "organization_id": "org-W

In [8]:
response_finetuned = openai.FineTuningJob.create(
    training_file=file_id,
    model="gpt-3.5-turbo",
    suffix="anote") 

job_id = response_finetuned['id']

print(f"Fine-tuning job created successfully with ID: {job_id}")

Fine-tuning job created successfully with ID: ftjob-Pkd9Q8ulAtB7EbbTc6aY58Gd


In [13]:
# Retrieve the state of a fine-tune
retrieve_job = openai.FineTuningJob.retrieve(job_id)
print(retrieve_job)


fine_tuned_model_id = retrieve_job["fine_tuned_model"]
print(f"Fine-tuned model id: {fine_tuned_model_id}")



{
  "object": "fine_tuning.job",
  "id": "ftjob-Pkd9Q8ulAtB7EbbTc6aY58Gd",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1696987687,
  "finished_at": 1696988031,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:personal:anote:88IOfgTc",
  "organization_id": "org-WM0C8DmpfiXb4ySIeUGFLcf2",
  "result_files": [
    "file-JgFfEm2h25wvVh3nELM9kcvX"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-G66jFKJJLR4twLoGEfQsTV9M",
  "hyperparameters": {
    "n_epochs": 10
  },
  "trained_tokens": 52840,
  "error": null
}
Fine-tuned model id: ft:gpt-3.5-turbo-0613:personal:anote:88IOfgTc


For future reference: model trained on sarcastic data has model id: ft:gpt-3.5-turbo-0613:personal:anote:85SGpKuh
Model trained on just apple dataset: ft:gpt-3.5-turbo-0613:personal:anote:88IOfgTc

In [4]:
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a factual chatbot that answers questions about 10-K documents. You only answer with answers you find in the text, no outside information."},
    {"role": "user", "content": "This is our information from the 10-K: Business Seasonality and Product IntroductionsThe Company has historically experienced higher net sales in its ﬁrst quarter compared to other quarters in its ﬁscal year due in part toseasonal holiday demand. Additionally, new product and service introductions can signiﬁcantly impact net sales, cost of sales andoperating expenses. The timing of product introductions can also impact the Company’s net sales to its indirect distribution channels asthese channels are ﬁlled with new inventory following a product launch, and channel inventory of an older product often declines as thelaunch of a newer product approaches. Net sales can also be affected when consumers and distributors anticipate a productintroduction.Human CapitalThe Company believes it has a talented, motivated and dedicated team, and works to create an inclusive, safe and supportiveenvironment for all of its team members. As of September 24, 2022, the Company had approximately 164,000 full-time equivalentemployees.Workplace Practices and PoliciesThe Company is an equal opportunity employer committed to inclusion and diversity and to providing a workplace free of harassment ordiscrimination.Compensation and BeneﬁtsThe Company believes that compensation should be competitive and equitable, and should enable employees to share in theCompany’s success. The Company recognizes its people are most likely to thrive when they have the resources to meet their needs andthe time and support to succeed in their professional and personal lives. In support of this, the Company offers a wide variety of beneﬁtsfor employees around the world and invests in tools and resources that are designed to support employees’ individual growth anddevelopment.Inclusion and DiversityThe Company remains committed to its vision to build and sustain a more inclusive workforce that is representative of the communities itserves. The Company continues to work to increase diverse representation at every level, foster an inclusive culture, and. Now, this is our question: Which quarter does Apple usually have higher net sales in?"}
  ]
)
print(completion.choices[0].message)

{
  "role": "assistant",
  "content": "According to the information provided in the 10-K, Apple typically experiences higher net sales in its first quarter compared to other quarters in its fiscal year due in part to seasonal holiday demand."
}


In [5]:
completion = openai.ChatCompletion.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are a factual chatbot that answers questions about 10-K documents. You only answer with answers you find in the text, no outside information."},
    {"role": "user", "content": "This is our information from the 10-K: Business Seasonality and Product IntroductionsThe Company has historically experienced higher net sales in its ﬁrst quarter compared to other quarters in its ﬁscal year due in part toseasonal holiday demand. Additionally, new product and service introductions can signiﬁcantly impact net sales, cost of sales andoperating expenses. The timing of product introductions can also impact the Company’s net sales to its indirect distribution channels asthese channels are ﬁlled with new inventory following a product launch, and channel inventory of an older product often declines as thelaunch of a newer product approaches. Net sales can also be affected when consumers and distributors anticipate a productintroduction.Human CapitalThe Company believes it has a talented, motivated and dedicated team, and works to create an inclusive, safe and supportiveenvironment for all of its team members. As of September 24, 2022, the Company had approximately 164,000 full-time equivalentemployees.Workplace Practices and PoliciesThe Company is an equal opportunity employer committed to inclusion and diversity and to providing a workplace free of harassment ordiscrimination.Compensation and BeneﬁtsThe Company believes that compensation should be competitive and equitable, and should enable employees to share in theCompany’s success. The Company recognizes its people are most likely to thrive when they have the resources to meet their needs andthe time and support to succeed in their professional and personal lives. In support of this, the Company offers a wide variety of beneﬁtsfor employees around the world and invests in tools and resources that are designed to support employees’ individual growth anddevelopment.Inclusion and DiversityThe Company remains committed to its vision to build and sustain a more inclusive workforce that is representative of the communities itserves. The Company continues to work to increase diverse representation at every level, foster an inclusive culture, and. Now, this is our question: Which quarter does Apple usually have higher net sales in?"}
  ]
)
print(completion.choices[0].message)

NameError: name 'fine_tuned_model_id' is not defined